<a href="https://colab.research.google.com/github/DavideBruni/CIDL_lab/blob/main/CIDL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision


train_mnist = torchvision.datasets.MNIST(
    "./data",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)

test_mnist = torchvision.datasets.MNIST(
    "./data",
    train=False,
    download=True,
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.1307,), (0.3081,))
    ])
)

# **Exercise 0**

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(28 * 28, 300),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(300, 300),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(300, 10),
    torch.nn.Softmax(dim=1)
)

model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=300, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=300, out_features=300, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=300, out_features=10, bias=True)
  (5): Softmax(dim=1)
)

In [ ]:
from tqdm import trange
from tqdm import trange, tqdm

dl = torch.utils.data.DataLoader(train_mnist, batch_size=32, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(3):
    bar = tqdm(dl)
    for digit,cls in bar:

        digit = digit.to("cuda")
        cls = cls.to('cuda')


        digit = digit.view(digit.shape[0], 28 * 28)
        cls_out = model(digit)

        loss = loss_fn(cls_out, cls)
        accuracy = (cls_out.argmax(dim=1) == cls).float().mean()
        bar.set_description(f"Loss: {loss.item():.4f}, Accuracy: {accuracy.item():.4f}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Loss: 1.4614, Accuracy: 1.0000: 100%|██████████| 1875/1875 [00:24<00:00, 77.87it/s]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

class DigitDataset(torch.utils.data.Dataset):

  def __init__(self,dataset):
    self.dataset = dataset

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,i):
    return self.dataset[i]

ds = DigitDataset(test_mnist)
dl_test = torch.utils.data.DataLoader(ds, batch_size=32)


correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculation for testing
    for digit, labels in dl_test:
        digit = digit.to("cuda")
        labels = labels.to("cuda")
        digit = digit.view(digit.shape[0], 28 * 28)  # Flatten the images if your model expects flat input
        outputs = model(digit)  # Forward pass
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')

Accuracy on the test dataset: 91.14%


In [ ]:
import random

class NumberPairs(torch.utils.data.IterableDataset):

  def __init__(self,dataset):
    super().__init__()
    self.dataset = dataset

  def __iter__(self):
    return self

  def __next__(self):
    a_digit, a_class = random.choice(self.dataset)
    b_digit, b_class = random.choice(self.dataset)

    return a_digit, b_digit, a_class, b_class

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,i):
    return self.dataset[i]

In [ ]:
from torch.nn.modules.activation import LeakyReLU


model = torch.nn.Sequential(
  torch.nn.Conv2d(1,48,(3,3), stride = 1, padding = 0),
  torch.nn.LeakyReLU(),
  torch.nn.Conv2d(48,48,(3,3), stride = 1, padding = 0),
  torch.nn.LeakyReLU(),
  torch.nn.Conv2d(48,10,(3,3), stride = 1, padding = 0),
  torch.nn.Flatten()
)

batches = 1000
ds = NumberPairs(train_mnist)
dl = torch.utils.data.DataLoader(ds, batch_size=32)

model.to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
dist_fn = torch.nn.PairwiseDistance(p=2)
loss_fn = torch.nn.MSELoss()

for epoch in range(3):
    bar = tqdm(enumerate(dl), total=batches)
    for i, (a_digits, b_digits, a_classes, b_classes) in bar:

        a_digits = a_digits.to("cuda")
        b_digits = b_digits.to("cuda")
        a_classes = a_classes.to("cuda")
        b_classes = b_classes.to("cuda")

        optimizer.zero_grad()
        a_outs = model(a_digits)
        b_outs = model(b_digits)

        dist = dist_fn(a_outs, b_outs)
        target_dists = (a_classes - b_classes).abs().float()

        loss = loss_fn(dist, target_dists)

        loss.backward()
        optimizer.step()

        bar.set_description(f"Loss {loss.item():.4f}")

        if i >= batches:
            break

Loss 3.9655: 100%|██████████| 1000/1000 [00:16<00:00, 60.13it/s]
